In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter1d
from scipy.signal import detrend


def find_peaks_original(x, scale=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Automatic Multi-Scale Peak Detection originally proposed in
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012, 5, 588-603
    https://doi.org/10.1109/ICRERA.2016.7884365
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    scale : int, optional
        specify maximum scale window size of (2 * scale + 1)
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        and scale with most local maxima, `l`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.zeros((L, N), dtype=bool)
    for k in np.arange(1, L):
        LSM[k - 1, k:N - k] = (
            (x[0:N - 2 * k] < x[k:N - k]) & (x[k:N - k] > x[2 * k:N])
        )

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, l_scale
    return pks


def find_peaks(x, scale=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using AMPD algorithm.
    Extended implementation handles peaks near start/end of the signal.
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    scale : int, optional
        specify maximum scale window size of (2 * scale + 1)
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        weigted number of maxima, 'G',
        and scale at which G is maximized, `l`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    L = N // 2
    if scale:
        L = min(scale, L)

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N]
                                )  # compare to right neighbours
        LSM[k - 1, k:N] &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Find scale with most maxima
    G = LSM.sum(axis=1)
    G = G * np.arange(
        N // 2, N // 2 - L, -1
    )  # normalize to adjust for new edge regions
    l_scale = np.argmax(G)

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM[0:l_scale, :], axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, LSM, G, l_scale
    return pks


def find_peaks_adaptive(x, window=None, debug=False):
    """Find peaks in quasi-periodic noisy signals using ASS-AMPD algorithm.
    Adaptive Scale Selection Automatic Multi-Scale Peak Detection,
    an extension of AMPD -
    "An Efficient Algorithm for Automatic Peak Detection in
    Noisy Periodic and Quasi-Periodic Signals", Algorithms 2012, 5, 588-603
    https://doi.org/10.1109/ICRERA.2016.7884365
    Optimized implementation by Igor Gotlibovych, 2018
    Parameters
    ----------
    x : ndarray
        1-D array on which to find peaks
    window : int, optional
        sliding window size for adaptive scale selection
    debug : bool, optional
        if set to True, return the Local Scalogram Matrix, `LSM`,
        and `adaptive_scale`,
        together with peak locations
    Returns
    -------
    pks: ndarray
        The ordered array of peak indices found in `x`
    """
    x = detrend(x)
    N = len(x)
    if not window:
        window = N
    if window > N:
        window = N
    L = window // 2

    # create LSM matix
    LSM = np.ones((L, N), dtype=bool)
    for k in np.arange(1, L + 1):
        LSM[k - 1, 0:N - k] &= (x[0:N - k] > x[k:N]
                                )  # compare to right neighbours
        LSM[k - 1, k:N] &= (x[k:N] > x[0:N - k])  # compare to left neighbours

    # Create continuos adaptive LSM
    ass_LSM = uniform_filter1d(LSM * window, window, axis=1, mode='nearest')
    normalization = np.arange(L, 0, -1)  # scale normalization weight
    ass_LSM = ass_LSM * normalization.reshape(-1, 1)

    # Find adaptive scale at each point
    adaptive_scale = ass_LSM.argmax(axis=0)

    # construct reduced LSM
    LSM_reduced = LSM[:adaptive_scale.max(), :]
    mask = (np.indices(LSM_reduced.shape)[0] > adaptive_scale
            )  # these elements are outside scale of interest
    LSM_reduced[mask] = 1

    # find peaks that persist on all scales up to l
    pks_logical = np.min(LSM_reduced, axis=0)
    pks = np.flatnonzero(pks_logical)
    if debug:
        return pks, ass_LSM, adaptive_scale
    return pks

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data as Data
import torchvision.datasets
import torchvision.transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob
# from pyampd.ampd import find_peaks


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

import matplotlib.pyplot as plt


import joblib

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip gdrive/My\ Drive/archive.zip

Archive:  gdrive/My Drive/archive.zip
replace Samples/rec_1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
PPG_datas = []
ABP_datas = []
ECG_datas = []
i = 0
for name in glob.glob('./Samples/*.csv'):
    raw_training_data = pd.read_csv(name, header=None)
    raw_training_data = np.array(raw_training_data)
    #print(raw_training_data.shape)
    PPG_data = raw_training_data[0]
    ABP_data = raw_training_data[1]
    ECG_data = raw_training_data[2]

    PPG_datas.append(PPG_data)
    ABP_datas.append(ABP_data)
    ECG_datas.append(ECG_data)
    # i = i + 1
 #   print(i)
#     if (i == 100):
#         break

PPG_datas = np.array(PPG_datas)
ABP_datas = np.array(ABP_datas)
ECG_datas = np.array(ECG_datas)
# raw_training_data.head()

In [ ]:
import pywt
from sklearn.decomposition import PCA

def generate_wavelet_vector(X, wavelet_family, decomposition_level):
    coefficients = pywt.wavedec(X, wavelet_family, level=decomposition_level)
    vector = np.array([])
    for coeffs in coefficients:
        vector = np.concatenate((vector, coeffs))
    return vector

In [ ]:
SAMPLE_FREQ = 125

In [ ]:
wavelet_family = 'db4'
decomposition_level = 5

wavelet_vectors = []
SBP_data = []
DBP_data = []
MAP_data = []

for j in range(len(PPG_datas)):
    sec_15 = 15 * SAMPLE_FREQ
    PPG_data = PPG_datas[j]
    ABP_data = ABP_datas[j]
    PPG_peaks = find_peaks(PPG_data, scale=SAMPLE_FREQ)
    for i in range(2, PPG_peaks.shape[0]):
        X = PPG_data[PPG_peaks[i - 1]:PPG_peaks[i]]
        if len(X) < SAMPLE_FREQ:
            wavelet_vector = generate_wavelet_vector(X, wavelet_family, decomposition_level)

            SBP = np.max(ABP_data[PPG_peaks[i - 1]:PPG_peaks[i - 1] + sec_15])
            DBP = np.min(ABP_data[PPG_peaks[i - 1]:PPG_peaks[i - 1] + sec_15])
            MAP = SBP / 3 + 2 * DBP / 3

            wavelet_vectors.append(wavelet_vector)
            SBP_data.append(SBP)
            DBP_data.append(DBP)
            MAP_data.append(MAP)

In [ ]:
wavelet_vectors = np.array(wavelet_vectors)
SBP_data = np.array(SBP_data)
DBP_data = np.array(DBP_data)
MAP_data = np.array(MAP_data)

print(wavelet_vectors.shape)
print(SBP_data.shape)

# Find the maximum length of the wavelet vectors
max_length = max([len(vector) for vector in wavelet_vectors])

# Pad the wavelet vectors with zeros to make them uniform length
padded_wavelet_vectors = []
for vector in wavelet_vectors:
    padded_vector = np.pad(vector, (0, max_length - len(vector)), mode='constant')
    padded_wavelet_vectors.append(padded_vector)

padded_wavelet_vectors = np.array(padded_wavelet_vectors)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=43)
pca_wavelet_vectors = pca.fit_transform(padded_wavelet_vectors)

print(pca_wavelet_vectors.shape)


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, SBP_train, SBP_test, DBP_train, DBP_test, MAP_train, MAP_test = train_test_split(
    pca_wavelet_vectors, SBP_data, DBP_data, MAP_data, test_size=0.1, random_state=42)

In [ ]:
def AAMI_standard(predict, test):
    total = len(predict)
    ME = np.mean(predict - test)
    MAE = np.mean(np.abs(predict-test))
    SD = np.std(predict-test)

    return total, ME, MAE, SD

def BHS_standard(predict, test):
    total = len(predict)
    mm5 = np.sum(np.abs(predict-test)<=5)
    mm10 = np.sum(np.abs(predict-test)<=10)
    mm15 = np.sum(np.abs(predict-test)<=15)
    return total, mm5, mm10, mm15

In [ ]:
EPOCHS = 2
LR = 0.001

In [ ]:
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, input_size):
        super(LSTM, self).__init__()
        self.hidden_dim = 256
        self.input = nn.LSTM(input_size=input_size, hidden_size=self.hidden_dim, num_layers=1, batch_first=True)
        self.hidden1 = nn.Linear(in_features=self.hidden_dim, out_features=512)
        self.hidden2 = nn.Linear(in_features=512, out_features=256)
        self.hidden3 = nn.Linear(in_features=256, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.dropout = nn.Dropout(p=0.2)
        self.activation = lambda X: F.relu(X)

    def forward(self, x):
        out, (h_n, c_n) = self.input(x)
        out = self.hidden1(self.activation(h_n[-1]))
        out = self.dropout(out)
        out = self.hidden2(self.activation(out))
        out = self.hidden3(self.activation(out))
        out = self.output(self.activation(out))
        return out

    def get_weight(self):
        weight = []
        weight.append(self.input.weight_ih_l0)
        weight.append(self.input.weight_hh_l0)
        weight.append(self.input.bias_ih_l0)
        weight.append(self.input.bias_hh_l0)
        weight.append(self.hidden1.weight)
        weight.append(self.hidden2.weight)
        weight.append(self.hidden3.weight)
        weight.append(self.output.weight)
        return weight

In [ ]:
# torch.set_default_tensor_type(torch.FloatTensor)

class LSTMModel():
    def __init__(self, net, train_loader, test_loader, EPOCH=20, LR=0.0001):
        self.net = net
        self.optimizer = torch.optim.Adam(net.parameters(), lr = LR)
        self.criterion = nn.MSELoss()
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.EPOCHS = EPOCH
        self.LR = LR

        self.net = self.net.to(device)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def start(self):

        self.history_train_loss = []
        self.history_test_loss = []
#         history_train_accuracy = []
#         history_test_accuracy = []
        for epoch in range(self.EPOCHS):
            print('Epoch:', epoch)
            train_loss = self.train()
            test_loss = self.test()

            self.history_train_loss.append(train_loss)
            self.history_test_loss.append(test_loss)
#             history_train_accuracy.append(train_accuracy)
#             history_test_accuracy.append(test_accuracy)

    def train(self):
        self.net.train()
        train_loss = 0
        for step, (batch_X, batch_y) in enumerate(self.train_loader):
            batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.net(batch_X).double()

            loss = self.criterion(outputs.reshape(-1), batch_y)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            #_, predicted = outputs.max(1)
            #total += batch_y.size(0)
            #correct += predicted.eq(batch_y).sum().item()

        print('【Training】Loss: %.3f ' % ( train_loss))
        return train_loss

    def test(self):
        self.net.eval()

        test_loss = 0
        with torch.no_grad():
            for step, (batch_X, batch_y) in enumerate(self.test_loader):
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                outputs = self.net(batch_X).double()
                loss = self.criterion(outputs.reshape(-1), batch_y)

                test_loss += loss.item()
#                 _, predicted = outputs.max(1)
#                 #print(predicted, batch_y)
#                 total += batch_y.size(0)
#                 correct += predicted.eq(batch_y).sum().item()

        print('【Testing】Loss: %.3f )' % ( test_loss))
        return test_loss

    def predict(self, test_loader):
        outputs = []
        self.net.eval()

        with torch.no_grad():
            for step, (batch_X, batch_y) in enumerate(test_loader):
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                output = self.net(batch_X).double().cpu().numpy().reshape(-1)
                for i in range(len(output)):
                    outputs.append(output[i])

        return outputs

    def get_weight(self):
        return self.net.get_weight()

    def get_model(self):
        return self.net.state_dict()

In [ ]:
from sklearn.model_selection import train_test_split

L = 43

x_train, x_test, SBP_train, SBP_test, DBP_train, DBP_test, MAP_train, MAP_test = train_test_split(
                                            pca_wavelet_vectors, SBP_data, DBP_data, MAP_data, test_size=0.1, random_state=42)


x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
SBP_train = torch.from_numpy(SBP_train)
SBP_test = torch.from_numpy(SBP_test)
DBP_train = torch.from_numpy(DBP_train)
DBP_test = torch.from_numpy(DBP_test)
MAP_train = torch.from_numpy(MAP_train)
MAP_test = torch.from_numpy(MAP_test)

x_train = x_train.view(-1, 1, L)
x_test = x_test.view(-1, 1, L)

print(x_train.shape, x_test.shape, SBP_train.shape, SBP_test.shape)

SBP_train_dataset = Data.TensorDataset(x_train, SBP_train)
SBP_test_dataset = Data.TensorDataset(x_test, SBP_test)


SBP_train_loader = Data.DataLoader(
    dataset = SBP_train_dataset,
    batch_size = 128,
)

SBP_test_loader = Data.DataLoader(
    dataset = SBP_test_dataset,
    batch_size = 10,
)

DBP_train_dataset = Data.TensorDataset(x_train, DBP_train)
DBP_test_dataset = Data.TensorDataset(x_test, DBP_test)


DBP_train_loader = Data.DataLoader(
    dataset = DBP_train_dataset,
    batch_size = 128,
)

DBP_test_loader = Data.DataLoader(
    dataset = DBP_test_dataset,
    batch_size = 10,
)

MAP_train_dataset = Data.TensorDataset(x_train, MAP_train)
MAP_test_dataset = Data.TensorDataset(x_test, MAP_test)


MAP_train_loader = Data.DataLoader(
    dataset = MAP_train_dataset,
    batch_size = 128,
)

MAP_test_loader = Data.DataLoader(
    dataset = MAP_test_dataset,
    batch_size = 10,
)

In [ ]:
SBP_lstm_module = LSTMModel(LSTM(input_size=L), SBP_train_loader, SBP_test_loader, EPOCH=200, LR=LR)
SBP_lstm_module.start()

In [ ]:
DBP_lstm_module = LSTMModel(LSTM(input_size=L), DBP_train_loader, DBP_test_loader, EPOCH=200, LR=LR)
DBP_lstm_module.start()

In [ ]:
MAP_lstm_module = LSTMModel(LSTM(input_size=L), MAP_train_loader, MAP_test_loader, EPOCH=200, LR=LR)
MAP_lstm_module.start()

In [ ]:
# temp_predict = []
# for predicts in SBP_predict:
#     for i in range(len(predicts))
#         temp_predict.append(predicts[i])
# print(SBP_predict[0:20])

# print(SBP_test[0:20])
import time

# Record the start time
start_time = time.time()

SBP_predict = np.array(SBP_lstm_module.predict(SBP_test_loader)).reshape(-1)
SBP_test = np.array(SBP_test)
DBP_predict = np.array(DBP_lstm_module.predict(DBP_test_loader)).reshape(-1)
DBP_test = np.array(DBP_test)
MAP_predict = np.array(MAP_lstm_module.predict(MAP_test_loader)).reshape(-1)
MAP_test = np.array(MAP_test)

print('LSTM')

total, ME, MAE, SD = AAMI_standard(SBP_predict, SBP_test)
print()
print()
print('----------------SBP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(SBP_predict, SBP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

total, ME, MAE, SD = AAMI_standard(DBP_predict, DBP_test)
print()
print()
print('----------------DBP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(DBP_predict, DBP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

total, ME, MAE, SD = AAMI_standard(MAP_predict, MAP_test)
print()
print()
print('----------------MAP-----------------')
print()
print('-------------AAMI standard----------')
print('ME             MAE             SD           total')
print('%.3f         %.3f         %.3f          %d' % (ME,MAE,SD,total))
#print('ME: ', ME, '   MAE: ', MAE, '   SD: ', SD, '   total: ', total)
total, mm5, mm10, mm15 = BHS_standard(MAP_predict, MAP_test)
print()
print('-------------BHS standard------------')
print('<5mmHg        <10mmHg        <15mmHg        total')
print('%d            %d           %d          %d' % (mm5,mm10,mm15,total))
print('%.3f%s        %.3f%s       %.3f%s         %d' % (mm5/total*100, '%',mm10/total*100, '%',mm15/total*100, '%', total))

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Time taken:", elapsed_time, "seconds")

In [ ]:
print('LSTM')

SBP_cc= np.corrcoef(SBP_predict, SBP_test)
DBP_cc= np.corrcoef(DBP_predict, DBP_test)
MAP_cc= np.corrcoef(MAP_predict, MAP_test)
print()
print()
print('------------Correlation Coefficient-------------')
print()
print('SBP: %.3f' % (SBP_cc[0, 1]))
print()
print('DBP: %.3f' % (DBP_cc[0, 1]))
print()
print('MAP: %.3f' % (MAP_cc[0, 1]))